In [1]:
from pyspark.sql import SparkSession

In [2]:
sparkCtx = SparkSession.builder.appName('null').getOrCreate()

24/05/03 15:43:15 WARN Utils: Your hostname, willians-desktop resolves to a loopback address: 127.0.1.1; using 192.168.1.73 instead (on interface enp1s0)
24/05/03 15:43:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 15:43:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = sparkCtx.read.csv('null_base.csv', inferSchema=True, header=True, sep=',')

In [5]:
df.columns

['id', 'nome', 'nascimento', 'idade', 'cidade']

In [18]:
df.describe().show()

+-------+------------------+----------+----------+-----------------+--------+
|summary|                id|      nome|nascimento|            idade|  cidade|
+-------+------------------+----------+----------+-----------------+--------+
|  count|                10|         7|         5|                6|       7|
|   mean|               4.9|      NULL|      NULL|             45.5|    NULL|
| stddev|3.4785054261852175|      NULL|      NULL|28.44468315872054|    NULL|
|    min|                 0|Bartolomeu|  01/01/11|               -1|Altamira|
|    max|                10|  Willians|  06/09/85|               80|  Uruará|
+-------+------------------+----------+----------+-----------------+--------+



In [19]:
df.na.drop().show()

+---+--------+----------+-----+----------+
| id|    nome|nascimento|idade|    cidade|
+---+--------+----------+-----+----------+
|  0|Willians|  06/08/94|   29|  Altamira|
|  9|   Pedro|  06/09/85|   50|Uberlandia|
+---+--------+----------+-----+----------+



In [20]:
df.na.drop(subset=['cidade']).show()

+---+---------+----------+-----+------------+
| id|     nome|nascimento|idade|      cidade|
+---+---------+----------+-----+------------+
|  0| Willians|  06/08/94|   29|    Altamira|
|  1|     NULL|  01/01/91|   -1|         Dds|
|  4|    Maria|      NULL|   65|      Uruará|
|  7|     NULL|  01/01/11|   80|Desconhecida|
|  8|        M|      NULL| NULL|           B|
|  9|    Pedro|  06/09/85|   50|  Uberlandia|
| 10|Marivaldo|      NULL| NULL|Medicilância|
+---+---------+----------+-----+------------+



In [34]:
# df.na.drop(how='all').show()
df.na.drop(how='any').show()

+---+--------+----------+-----+----------+
| id|    nome|nascimento|idade|    cidade|
+---+--------+----------+-----+----------+
|  0|Willians|  06/08/94|   29|  Altamira|
|  9|   Pedro|  06/09/85|   50|Uberlandia|
+---+--------+----------+-----+----------+



In [35]:
df.na.fill(0, subset='idade').show()

+---+----------+----------+-----+------------+
| id|      nome|nascimento|idade|      cidade|
+---+----------+----------+-----+------------+
|  0|  Willians|  06/08/94|   29|    Altamira|
|  1|      NULL|  01/01/91|   -1|         Dds|
|  2|Bartolomeu|      NULL|    0|        NULL|
|  3|    Estela|  01/08/98|   50|        NULL|
|  4|     Maria|      NULL|   65|      Uruará|
|  5|      NULL|      NULL|    0|        NULL|
|  7|      NULL|  01/01/11|   80|Desconhecida|
|  8|         M|      NULL|    0|           B|
|  9|     Pedro|  06/09/85|   50|  Uberlandia|
| 10| Marivaldo|      NULL|    0|Medicilância|
+---+----------+----------+-----+------------+



In [36]:
df.na.fill('Vazio').show()

+---+----------+----------+-----+------------+
| id|      nome|nascimento|idade|      cidade|
+---+----------+----------+-----+------------+
|  0|  Willians|  06/08/94|   29|    Altamira|
|  1|     Vazio|  01/01/91|   -1|         Dds|
|  2|Bartolomeu|     Vazio| NULL|       Vazio|
|  3|    Estela|  01/08/98|   50|       Vazio|
|  4|     Maria|     Vazio|   65|      Uruará|
|  5|     Vazio|     Vazio| NULL|       Vazio|
|  7|     Vazio|  01/01/11|   80|Desconhecida|
|  8|         M|     Vazio| NULL|           B|
|  9|     Pedro|  06/09/85|   50|  Uberlandia|
| 10| Marivaldo|     Vazio| NULL|Medicilância|
+---+----------+----------+-----+------------+



In [4]:
from pyspark.sql.functions import mean

In [18]:
mean_val = df.select(mean(df['idade'])).collect()[0][0]
df.na.fill(mean_val, subset=['idade']).show()

+---+----------+----------+-----+------------+
| id|      nome|nascimento|idade|      cidade|
+---+----------+----------+-----+------------+
|  0|  Willians|08/06/1994|   29|    Altamira|
|  1|      NULL|01/01/1991|   -1|         Dds|
|  2|Bartolomeu|      NULL|   45|        NULL|
|  3|    Estela|01/08/1998|   50|        NULL|
|  4|     Maria|      NULL|   65|      Uruará|
|  5|      NULL|      NULL|   45|        NULL|
|  7|      NULL|01/01/1911|   80|Desconhecida|
|  8|         M|      NULL|   45|           B|
|  9|     Pedro|06/09/1985|   50|  Uberlandia|
| 10| Marivaldo|      NULL|   45|Medicilância|
+---+----------+----------+-----+------------+



In [35]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date, year
df_date = df.na.fill('1999-01-01')
df_date = df_date.withColumn('Year', year(to_date(df_date['nascimento'])))
df_date.groupBy('Year').mean().select(['Year', 'avg(idade)']).withColumnRenamed('avg(idade)', 'Idade (avg)').show()

+----+-----------+
|Year|Idade (avg)|
+----+-----------+
|NULL|       41.6|
|1999|       65.0|
+----+-----------+

